In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector as selector
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('cycles.csv')
df

,OriginalVariableNames,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var394,Var395,Var396,Var397,Var398,Var399,Var400,Var401,Var402,Var403
0,cherry10 1,166,172,163,161,171,171,184,163,174,...,163,168,167,166,162,166,163,cherry,10,cherry10
1,cherry10 2,156,161,152,161,158,158,165,159,160,...,162,153,166,161,175,168,165,cherry,10,cherry10
2,cherry10 3,163,159,156,156,163,167,171,164,157,...,167,156,161,146,170,166,169,cherry,10,cherry10
3,cherry10 4,149,157,131,158,149,158,143,150,133,...,153,151,155,149,160,147,153,cherry,10,cherry10
4,cherry10 5,136,150,145,146,130,141,142,141,138,...,150,147,152,146,143,141,150,cherry,10,cherry10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,acrylic100 26,106,110,121,121,118,103,121,109,103,...,106,115,107,118,118,113,99,acrylic,100,acrylic100
1422,acrylic100 27,112,115,119,117,112,116,123,106,104,...,118,108,130,96,113,113,112,acrylic,100,acrylic100
1423,acrylic100 28,111,104,117,121,121,115,118,124,133,...,109,123,116,116,107,121,102,acrylic,100,acrylic100
1424,acrylic100 29,116,114,121,124,112,122,109,114,110,...,121,114,117,107,110,101,112,acrylic,100,acrylic100


In [213]:
#create tables for two different predictors
first_pass = df.drop(columns=['Var402', 'Var403', 'OriginalVariableNames'])
second_pass = df.drop(columns=['Var403', 'OriginalVariableNames'])

In [214]:
#parse first pass table into test and train
trainPt1 = first_pass.sample(frac = 0.75)

testPt1 = df.drop(trainPt1.index)
trainPt1

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var392,Var393,Var394,Var395,Var396,Var397,Var398,Var399,Var400,Var401
247,79,78,76,103,92,94,80,73,70,82,...,94,81,76,73,91,82,81,79,81,cherry
1118,118,108,114,117,108,116,122,119,120,118,...,120,118,117,106,127,103,115,116,113,acrylic
1050,136,125,126,122,111,135,110,135,124,120,...,127,127,125,130,131,116,121,127,123,acrylic
1076,131,126,121,117,121,115,121,121,126,120,...,117,126,115,131,113,132,116,128,122,acrylic
850,108,106,93,99,101,99,105,95,87,103,...,103,104,92,94,104,102,108,99,110,basswood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,128,106,107,116,110,123,114,121,119,118,...,100,117,104,122,112,129,110,96,108,acrylic
1090,137,131,143,130,131,130,142,123,132,130,...,126,138,129,132,132,138,132,124,152,acrylic
469,80,80,73,81,78,92,84,73,96,92,...,77,75,80,69,77,66,77,75,67,cherry
137,85,87,75,86,97,88,90,94,98,96,...,82,98,98,90,94,94,87,92,96,cherry


In [215]:
y = trainPt1.iloc[:,400]
features = list(trainPt1.columns)
features.pop()
X = trainPt1[features]
X.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var391,Var392,Var393,Var394,Var395,Var396,Var397,Var398,Var399,Var400
247,79,78,76,103,92,94,80,73,70,82,...,77,94,81,76,73,91,82,81,79,81
1118,118,108,114,117,108,116,122,119,120,118,...,105,120,118,117,106,127,103,115,116,113
1050,136,125,126,122,111,135,110,135,124,120,...,134,127,127,125,130,131,116,121,127,123
1076,131,126,121,117,121,115,121,121,126,120,...,129,117,126,115,131,113,132,116,128,122
850,108,106,93,99,101,99,105,95,87,103,...,108,103,104,92,94,104,102,108,99,110


In [216]:
#validation data, somewhat unneccesary
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
accuracy_score(val_y, rf_val_predictions)

0.8507462686567164

In [217]:
Xfinal = first_pass[features]
yfinal = first_pass.iloc[:,400]
cleanRegressor = RandomForestClassifier(random_state=1)
train_X, test_X, train_y, test_y = train_test_split(Xfinal, yfinal, random_state=1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(test_X)
accuracy_score(test_y, rf_val_predictions)

0.834733893557423

In [218]:
#predicting speed assuming categorical prediction was correct
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

data = second_pass.iloc[: , :-1]
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

In [219]:
y = second_pass.iloc[:,401]
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])
rf_regressor = RandomForestRegressor(random_state=1)
model = make_pipeline(preprocessor, rf_regressor)
X = data
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=1)
model.fit(train_X, train_y)
rf_predictions = model.predict(test_X)
rf_val_mae = mean_absolute_error(rf_predictions, test_y)
rf_val_mae

14.8609243697479

In [220]:
df

,OriginalVariableNames,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var394,Var395,Var396,Var397,Var398,Var399,Var400,Var401,Var402,Var403
0,cherry10 1,166,172,163,161,171,171,184,163,174,...,163,168,167,166,162,166,163,cherry,10,cherry10
1,cherry10 2,156,161,152,161,158,158,165,159,160,...,162,153,166,161,175,168,165,cherry,10,cherry10
2,cherry10 3,163,159,156,156,163,167,171,164,157,...,167,156,161,146,170,166,169,cherry,10,cherry10
3,cherry10 4,149,157,131,158,149,158,143,150,133,...,153,151,155,149,160,147,153,cherry,10,cherry10
4,cherry10 5,136,150,145,146,130,141,142,141,138,...,150,147,152,146,143,141,150,cherry,10,cherry10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,acrylic100 26,106,110,121,121,118,103,121,109,103,...,106,115,107,118,118,113,99,acrylic,100,acrylic100
1422,acrylic100 27,112,115,119,117,112,116,123,106,104,...,118,108,130,96,113,113,112,acrylic,100,acrylic100
1423,acrylic100 28,111,104,117,121,121,115,118,124,133,...,109,123,116,116,107,121,102,acrylic,100,acrylic100
1424,acrylic100 29,116,114,121,124,112,122,109,114,110,...,121,114,117,107,110,101,112,acrylic,100,acrylic100


In [221]:
#concerns: only three materials, so predictor is likely pretty inflated
df.iloc[:,401].unique()

array(['cherry', 'basswood', 'acrylic'], dtype=object)

In [222]:
rnge = df.iloc[:,402].max() - df.iloc[:,402].min()
rf_val_mae / rnge

0.16512138188608777

In [223]:
#works for table where last column is what we want to predict
def category_accuracy(tbl):
    y = tbl.iloc[:, -1]
    features = list(tbl.columns)
    features.pop()
    X = tbl[features]
    y = tbl.iloc[:,400]
    model = RandomForestClassifier()
    train_X, test_X, train_y, test_y = train_test_split(Xfinal, yfinal, random_state=1)
    model.fit(train_X, train_y)
    rf_val_predictions = model.predict(test_X)
    return accuracy_score(test_y, rf_val_predictions)

In [224]:
category_accuracy(first_pass)

0.84593837535014

In [225]:
def speed_accuracy(tbl):
    data = tbl.iloc[: , :-1]
    numerical_columns = numerical_columns_selector(data)
    categorical_columns = categorical_columns_selector(data)
    y = tbl.iloc[:,401]
    preprocessor = ColumnTransformer([
        ('one-hot-encoder', categorical_preprocessor, categorical_columns),
        ('standard_scaler', numerical_preprocessor, numerical_columns)])
    rf_regressor = RandomForestRegressor()
    model = make_pipeline(preprocessor, rf_regressor)
    X = data
    train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=1)
    model.fit(train_X, train_y)
    rf_predictions = model.predict(test_X)
    rf_val_mae = mean_absolute_error(rf_predictions, test_y)
    return rf_val_mae

In [227]:
speed_accuracy(second_pass)

14.859103641456581

In [236]:
def category_accuracy_cross_validation(tbl):
    y = tbl.iloc[:, -1]
    features = list(tbl.columns)
    features.pop()
    X = tbl[features]
    y = tbl.iloc[:,400]
    model = RandomForestClassifier()
    train_X, test_X, train_y, test_y = train_test_split(Xfinal, yfinal, random_state=1)
    model.fit(train_X, train_y)
#     rf_val_predictions = model.predict(test_X)
    scores = cross_val_score(model, X, y,
                              cv=5,
                              scoring='accuracy')
    return np.mean(scores)

In [237]:
category_accuracy_cross_validation(first_pass)

0.8185007974481658

In [238]:
def speed_accuracy_cross(tbl):
    data = tbl.iloc[: , :-1]
    numerical_columns = numerical_columns_selector(data)
    categorical_columns = categorical_columns_selector(data)
    y = tbl.iloc[:,401]
    preprocessor = ColumnTransformer([
        ('one-hot-encoder', categorical_preprocessor, categorical_columns),
        ('standard_scaler', numerical_preprocessor, numerical_columns)])
    rf_regressor = RandomForestRegressor()
    model = make_pipeline(preprocessor, rf_regressor)
    X = data
    train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=1)
    model.fit(train_X, train_y)
#     rf_predictions = model.predict(test_X)
    scores = -1 * cross_val_score(model, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
    return np.mean(scores)

In [241]:
err = speed_accuracy_cross(second_pass)

In [242]:
err / rnge

0.3557030875557191

In [243]:
err

32.01327788001472